In [1]:
import numpy as np

matmul_call_count = 0 # A counter to keep track of matmul calls

def reset_matmul_call_counter():
    global matmul_call_count
    matmul_call_count = 0

def matmul_counter_wrapper(func):
    def wrapper(*args, **kwargs):
        global matmul_call_count
        matmul_call_count += 1
        return func(*args, **kwargs)
    return wrapper

# Replace np.matmul with the logged version
np.matmul = matmul_counter_wrapper(np.matmul)

class ag: # AutoGrad
    """
    A barebone version of the AutoGrad library that we've been working with
    it only supports 
    - matmul 
        - between two numpy matrices ONLY,
            -i.e., arrays X such that len(X.shape) == 2
    - sum 
        - "axis = None" ONLY
            - so everything gets summed up
    - add (entrywise)
    """

    #################
    # REDUCTIVE OPS #
    #################
    def sum(input):
        output = ag.Tensor(np.sum(input.value), inputs = [input], op='sum')
        def _backward():
            if input.grad is None:
                input.zero_grad()
            input.grad += output.grad
            if not output.requires_grad:
                output.discard_grad()
            return None
        output._backward = _backward
        return output

    ##########
    # MATMUL #
    ##########
    def matmul(input1, input2):
        return input1@input2

    class Tensor: # Tensor with grads
        def __init__(self,
                     value,
                     requires_grad=False,
                     op="",
                     _backward= lambda : None,
                     inputs=[],
                     label=""):

            if type(value) in [float ,int]:
                value = np.array(value)
            
            self.requires_grad = requires_grad
            
            self.value = 1.0*value
            self.grad = None

            
            if self.requires_grad:
                self.grad = np.zeros_like(self.value)

            self.shape = value.shape

            self._backward = _backward
            self.inputs = inputs

            self.op = op
            self.label = label

        def topological_sort(self):
            topo_order = []
            visited = set()

            def dfs(node):
                if node not in visited:
                    visited.add(node)
                    for input in node.inputs:
                        dfs(input)
                    topo_order.append(node)

            dfs(self)
            return topo_order


        def backward(self):
            self.grad = np.array(1.0)

            
            topo_order = self.topological_sort()
            
            start_trace()  # added to trace memory used
            mem_usage = [] # added to trace memory used

            for node in reversed(topo_order):
                node._backward()
                mem_usage.append(snapshot_trace())
            end_trace()
            return mem_usage

        ##########
        # MATMUL #
        ##########
        def __matmul__(self,other):
            """
            matrix multiplication between two MATRICES only
            """

            assert(len(self.shape) == 2)
            assert(len(other.shape) == 2)

            output = ag.Tensor(np.matmul(self.value,other.value),
                               inputs = [self,other],
                               op="matmul")
            
            def _backward():
                if self.grad is None:
                    self.zero_grad()
                if other.grad is None:
                    other.zero_grad()
                    
                self.grad += np.matmul(output.grad, other.value.T)
                other.grad += np.matmul(self.value.T, output.grad)

                if not output.requires_grad:
                    output.discard_grad()
                return None
            output._backward = _backward
            return output
        
        def zero_grad(self):
            self.grad = np.zeros_like(self.value)
            return None
        
        def discard_grad(self):
            self.grad = None
            return None
            
        def __repr__(self) -> str:
            return "Value:\n"+self.value.__repr__() + "\nGrad:\n" + self.grad.__repr__()


In [2]:
import tracemalloc
import numpy as np
# code adapted from 
# https://numpy.org/doc/2.0/reference/c-api/data_memory.html#example-of-memory-tracing-with-np-lib-tracemalloc-domain

def start_trace():
    tracemalloc.start()
    return None

def snapshot_trace():
    snapshot = tracemalloc.take_snapshot()

    # only keep track of the allocations by numpy
    dom_filter = tracemalloc.DomainFilter(inclusive=True,
                                          domain=np.lib.tracemalloc_domain)
    
    snapshot = snapshot.filter_traces([dom_filter])
    top_stats = snapshot.statistics('traceback')

    return top_stats
    
def end_trace():
    
    tracemalloc.clear_traces()
    tracemalloc.stop()
    return None
    
def print_trace_stats(stats):
    mem_allocated = 0
    for stat in stats:
        mem_allocated += stat.size
    print(f"memory allocated: {mem_allocated//  1000000} MB")
    return None

In [3]:
np.random.seed(42)

num_layers = 10
num_samples = 4096
dim_hidden = 1000

weights = [ag.Tensor(0.02*np.random.randn(dim_hidden, dim_hidden), 
                     requires_grad = True) for _ in range(num_layers)]
X = ag.Tensor(np.random.randn(num_samples, dim_hidden))

def forward(x, weights):
    for w in weights:
        x = ag.matmul(x, w)
    return ag.sum(x)

def forward_traced_grad_discard(x, weights):
    start_trace()
    mem_usage = []
    for w in weights:
        x = ag.matmul(x, w)
        mem_usage.append(snapshot_trace())
    l = ag.sum(x)
    mem_usage.append(snapshot_trace())
    end_trace()
    return l, mem_usage

reset_matmul_call_counter()
l, mem_usage_forward = forward_traced_grad_discard(X,weights)
mem_usage_backward = l.backward()
print(matmul_call_count)

30


In [4]:
print( f"size of the numpy array {( dim_hidden * num_samples * 8 ) // 1000000} MB")

size of the numpy array 32 MB


In [5]:
for i, trace_stats in enumerate(mem_usage_forward):
    print(f"layer {i}")
    print_trace_stats(trace_stats)
# expected output
# layer 0
# memory allocated: 32 MB
# layer 1
# memory allocated: 65 MB
# layer 2
# memory allocated: 98 MB
# ...

layer 0
memory allocated: 32 MB
layer 1
memory allocated: 65 MB
layer 2
memory allocated: 98 MB
layer 3
memory allocated: 131 MB
layer 4
memory allocated: 163 MB
layer 5
memory allocated: 196 MB
layer 6
memory allocated: 229 MB
layer 7
memory allocated: 262 MB
layer 8
memory allocated: 294 MB
layer 9
memory allocated: 327 MB
layer 10
memory allocated: 327 MB


In [6]:
for i, trace_stats in enumerate(mem_usage_backward):
    print(f"backward step {i}")
    print_trace_stats(trace_stats)

# expected output
# backward step 0
# memory allocated: 32 MB
# backward step 1
# memory allocated: 32 MB
# backward step 2
# memory allocated: 32 MB
# ...

backward step 0
memory allocated: 32 MB
backward step 1
memory allocated: 32 MB
backward step 2
memory allocated: 32 MB
backward step 3
memory allocated: 32 MB
backward step 4
memory allocated: 32 MB
backward step 5
memory allocated: 32 MB
backward step 6
memory allocated: 32 MB
backward step 7
memory allocated: 32 MB
backward step 8
memory allocated: 32 MB
backward step 9
memory allocated: 32 MB
backward step 10
memory allocated: 32 MB
backward step 11
memory allocated: 32 MB
backward step 12
memory allocated: 32 MB
backward step 13
memory allocated: 32 MB
backward step 14
memory allocated: 32 MB
backward step 15
memory allocated: 32 MB
backward step 16
memory allocated: 32 MB
backward step 17
memory allocated: 32 MB
backward step 18
memory allocated: 32 MB
backward step 19
memory allocated: 32 MB
backward step 20
memory allocated: 32 MB
backward step 21
memory allocated: 32 MB


In [7]:
weights[0].grad
# EXPECTED OUTPUT

# array([[ 0.12749943, -0.28241071, -0.05621888, ..., -0.02374291,
#          0.27343724,  0.48198191],
#        [ 0.25550796, -0.56594908, -0.11266225, ..., -0.04758063,
#          0.5479663 ,  0.96588835],
# ...

array([[ 0.12749943, -0.28241071, -0.05621888, ..., -0.02374291,
         0.27343724,  0.48198191],
       [ 0.25550796, -0.56594908, -0.11266225, ..., -0.04758063,
         0.5479663 ,  0.96588835],
       [ 0.39728201, -0.87997801, -0.1751753 , ..., -0.07398176,
         0.85201711,  1.50183213],
       ...,
       [-0.0141211 ,  0.03127818,  0.00622648, ...,  0.00262963,
        -0.03028433, -0.05338153],
       [ 0.16727405, -0.37051134, -0.07375688, ..., -0.03114974,
         0.35873851,  0.63234061],
       [-0.29072346,  0.64395127,  0.12818998, ...,  0.05413846,
        -0.62349001, -1.09901236]])